In [1]:
from collections import defaultdict
from aocutils.grid import gridneigh, neighbors


grid, neigh = gridneigh(
    "in.txt", diag=True, to_dict=True, parser=lambda x: [ch for ch in x])
grid = {k for k, v in grid.items() if v == "#"}

N = [(-1, 0), (-1, 1), (-1, -1)]
S = [(1, 0), (1, 1), (1, -1)]
W = [(0, -1), (1, -1), (-1, -1)]
E = [(0, 1), (-1, 1), (1, 1)]
orderpnt = 0
order = [N, S, W, E]
grids = []
for step in range(1, 1000000):
    grids.append(grid)
    moved = False
    newgrid = defaultdict(set)
    nextround = set()

    for elf in grid:
        if any(n in grid for n in neighbors(elf, diag=True)):
            r, c = elf
            for i in range(4):
                o = order[(orderpnt + i) % 4]
                found = False
                for direc in o:
                    if (r + direc[0], c + direc[1]) in grid:
                        found = True
                        break
                if not found:
                    newgrid[(r + o[0][0], c + o[0][1])].add(elf)
                    break
            else:
                nextround.add(elf)
        else:
            nextround.add(elf)

    for newloc, elves in newgrid.items():
        if len(elves) > 1:
            for elf in elves:
                nextround.add(elf)
        else:
            nextround.add(tuple(newloc))
            moved = True
    orderpnt += 1
    
    grid = nextround

    if not moved:
        print("p2", step)
        break
    if step == 10:
        rows = [elf[0] for elf in grid]
        cols = [c for (r, c) in grid]
        print(
            "p1", (max(rows) - min(rows) + 1) * (max(cols) - min(cols) + 1) - len(grid)
        )

p1 4082
p2 1065


In [ ]:
from matplotlib import cm
import matplotlib.pyplot as plt
from itertools import chain
import numpy as np
from ipywidgets import interact

def animate(grids):
    minn = min(chain(*[cell for grid in grids for cell in grid]))
    maxx = max(chain(*[cell for grid in grids for cell in grid]))
    size = maxx-minn + 1
    arrs = []
    for grid in grids:
        arr = np.zeros((size,size))
        for r,c in grid:
            arr[r-minn,c-minn] = 1
        arrs.append(np.uint8(cm.viridis(arr)*255))
        
    @interact(frame=(0,len(arrs)-1))
    def show_frame(frame=0):
        plt.imshow(arrs[frame])
animate(grids)

interactive(children=(IntSlider(value=0, description='frame', max=1064), Output()), _dom_classes=('widget-inte…

In [ ]:
# below didn't really work with making mp4 video, colors & resolution sucks
import ffmpeg
import io


def vidwrite(fn, images, framerate=1, vcodec='libx264'):
    if not isinstance(images, np.ndarray):
        images = np.asarray(images)
    _,height,width,channels = images.shape
    process = (
        ffmpeg
            .input('pipe:', format='rawvideo', pix_fmt='rgba', s='{}x{}'.format(width, height), r=framerate)
            # .filter('scale', size='hd1080', force_original_aspect_ratio='increase')
            .output(fn, pix_fmt='yuv420p', crf =18 , vcodec=vcodec, r=framerate)
            .overwrite_output()
            .run_async(pipe_stdin=True, overwrite_output=True, pipe_stderr=True)
    )
    for frame in images:
        try:
            process.stdin.write(
                frame.astype(np.uint8).tobytes()
            )
        except Exception as e: # should probably be an exception related to process.stdin.write
            for line in io.TextIOWrapper(process.stderr, encoding="utf-8"): # I didn't know how to get the stderr from the process, but this worked for me
                print(line) # <-- print all the lines in the processes stderr after it has errored
            process.stdin.close()
            process.wait()
            return # cant run anymore so end the for loop and the function execution

In [ ]:
import ffmpeg
import io
from matplotlib import cm
import matplotlib.pyplot as plt

import numpy as np
from itertools import chain

minn = min(chain(*[cell for grid in grids for cell in grid]))
maxx = max(chain(*[cell for grid in grids for cell in grid]))
size = maxx-minn + 1
from PIL import Image
arrs = []
test2 = []
for grid in grids:

    arr = np.zeros((size,size))
    # norm = plt.Normalize()
    # colors = plt.cm.jet(norm(dz))
    for r,c in grid:
        arr[r-minn,c-minn] = 1
    # im = Image.fromarray()
    test2.append(arr)
    arrs.append(np.uint8(cm.viridis(arr)*255))
    

test = np.asarray(arrs)
# test = test[..., np.newaxis]
vidwrite('teting.mp4', test)